In [1]:
import warnings
warnings.filterwarnings( 'ignore' )

In [2]:
import pandas as  pd
import numpy as np
import pickle
import joblib
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

In [3]:
import variables_nombres as vn
import funciones as fun

# 1. Datos del periodo 17 - 19

## 1.1. Cargar los datos

In [4]:
path_1719 = r'..\..\input\vl\built_data\base17_19_anualizada.dta'
data_1719 = pd.read_stata( path_1719, 
                           convert_categoricals = False )

In [5]:
data_1719.shape

(74746, 188)

In [6]:
# Calcular número de personas por hogar
data_1719[ 'n_personas_hogar' ] = data_1719.groupby( [ 'conglome', 'vivienda', 'hogar' ] )[ 'codperso' ].transform( 'count' )

## 1.2. Calcular el porcentaje de valores perdidos

In [7]:
missing_percentage_1719 = ( data_1719.isnull().sum() / len( data_1719 ) ) * 100

wb = Workbook()
ws = wb.active

for col_idx, col_name in enumerate( data_1719.columns, start = 1 ):
    ws.cell( row = col_idx, 
             column = 1, 
             value = col_name )

for row_idx, percentage in enumerate( missing_percentage_1719, start = 1 ):
    ws.cell( row = row_idx, 
             column = 2, 
             value = percentage )

wb.save( r'..\..\input\vl\built_data\missing_values_percentaje_1719.xlsx' )

## 1.3. Imputar con media y con moda

In [8]:
data_1719 = fun.imputar_ii( data_1719, vn.num_vars, num = True, dummy = False )
data_1719 = fun.imputar_ii( data_1719, vn.categ_vars, num = False, dummy = False )

## 1.4. Binarizar variables multicategóricas

In [9]:
subset_binarizado        = []
variables_para_binarizar = [ var for var in vn.variables_para_binarizar if var in data_1719.columns ]
for col in variables_para_binarizar: 
    columna_binarizada   = pd.get_dummies( data_1719[ col ],
                                           prefix = col )
    subset_binarizado.append( columna_binarizada )
    
data_1719 = pd.concat( [ data_1719 ] + subset_binarizado, 
                       axis = 1 )
data_1719 = data_1719.drop( columns = variables_para_binarizar )

## 1.5. Colapsar

In [10]:
columnas_binarizadas = []
for df in subset_binarizado:
    columnas_binarizadas.extend( df.columns.tolist() )  

In [11]:
dummy_vars     = vn.variables_para_colapsar_dummies + columnas_binarizadas
dummy_vars     = [ var for var in dummy_vars if var not in vn.variables_para_binarizar and var in data_1719.columns ]
gasto_vars     = [ 'gasto_monetario_deflactado', 'gasto_bruto_deflactado', 'gashog1d', 'gashog2d', 'region', 'n_personas_hogar' ]
household_vars = [ var for var in vn.variables_nivel_hogar if var not in vn.no_colapsables_vars and var in data_1719.columns ]
household_vars = household_vars + gasto_vars
mean_vars      = [ var for var in vn.variables_para_colapsar_prom if var in data_1719.columns ]

In [12]:
agg_dict = { **{ col: 'max' for col in dummy_vars }, 
             **{ col: 'mean' for col in mean_vars },
             **{ col: 'first' for col in household_vars } }
data_1719 = data_1719.groupby( [ 'conglome', 'vivienda', 'hogar' ] ).agg( agg_dict ).reset_index()

In [13]:
data_1719[ 'gasto_monetario_deflactado_pc_m' ] = ( ( data_1719[ 'gasto_monetario_deflactado' ]/data_1719[ 'n_personas_hogar' ] ) / 12 ) * 100
data_1719[ 'gasto_bruto_deflactado_pc_m' ]     = ( ( data_1719[ 'gasto_bruto_deflactado' ]/data_1719[ 'n_personas_hogar' ] ) / 12 ) * 100
data_1719[ 'gashog1d_pc_m' ]                   = ( data_1719[ 'gashog1d' ]/data_1719[ 'n_personas_hogar' ] ) / 12
data_1719[ 'gashog2d_pc_m' ]                   = ( data_1719[ 'gashog2d' ]/data_1719[ 'n_personas_hogar' ] ) / 12

## 1.7. Exportar datos

In [14]:
data_1719.to_csv( r'..\..\input\vl\preprocessed_data\base17_19_anualizada_hogares.csv', 
                  index = False )

# 2. Datos del periodo 19 - 21

## 2.1. Cargar los datos

In [15]:
path_1921 = r'..\..\input\vl\built_data\base19_21_anualizada.dta'
data_1921 = pd.read_stata( path_1921, 
                           convert_categoricals = False )

In [16]:
data_1921.shape

(75072, 140)

In [17]:
# Calcular número de personas por hogar
data_1921[ 'n_personas_hogar' ] = data_1921.groupby( [ 'conglome', 'vivienda', 'hogar' ] )[ 'codperso' ].transform( 'count' )

## 2.2. Calcular el porcentaje de valores perdidos

In [18]:
missing_percentage_1921 = ( data_1921.isnull().sum() / len( data_1921 ) ) * 100

wb = Workbook()
ws = wb.active

for col_idx, col_name in enumerate( data_1921.columns, start = 1 ):
    ws.cell( row = col_idx, 
             column = 1, 
             value = col_name )

for row_idx, percentage in enumerate( missing_percentage_1921, start = 1 ):
    ws.cell( row = row_idx, 
             column = 2, 
             value = percentage )

wb.save( r'..\..\input\vl\built_data\missing_values_percentaje_1921.xlsx' )

## 2.3. Imputar con media y con moda

In [19]:
data_1921 = fun.imputar_ii( data_1921, vn.num_vars, num = True, dummy = False )
data_1921 = fun.imputar_ii( data_1921, vn.categ_vars, num = False, dummy = False )

## 1.4. Binarizar variables multicategóricas

In [20]:
subset_binarizado        = []
variables_para_binarizar = [ var for var in vn.variables_para_binarizar if var in data_1921.columns ]
for col in variables_para_binarizar: 
    columna_binarizada   = pd.get_dummies( data_1921[ col ],
                                           prefix = col )
    subset_binarizado.append( columna_binarizada )
    
data_1921 = pd.concat( [ data_1921 ] + subset_binarizado, 
                       axis = 1 )
data_1921 = data_1921.drop( columns = variables_para_binarizar )

## 1.5. Colapsar

In [21]:
columnas_binarizadas = []
for df in subset_binarizado:
    columnas_binarizadas.extend( df.columns.tolist() )  

In [22]:
dummy_vars     = vn.variables_para_colapsar_dummies + columnas_binarizadas
dummy_vars     = [ var for var in dummy_vars if var not in vn.variables_para_binarizar and var in data_1921.columns ]
gasto_vars     = [ 'gasto_monetario_deflactado', 'gasto_bruto_deflactado', 'gashog1d', 'gashog2d', 'region', 'n_personas_hogar' ]
household_vars = [ var for var in vn.variables_nivel_hogar if var not in vn.no_colapsables_vars and var in data_1921.columns ]
household_vars = household_vars + gasto_vars
mean_vars      = [ var for var in vn.variables_para_colapsar_prom if var in data_1921.columns ]

In [23]:
agg_dict = { **{ col: 'max' for col in dummy_vars }, 
             **{ col: 'mean' for col in mean_vars },
             **{ col: 'first' for col in household_vars } }
data_1921 = data_1921.groupby( [ 'conglome', 'vivienda', 'hogar' ] ).agg( agg_dict ).reset_index()

In [24]:
data_1921[ 'gasto_monetario_deflactado_pc_m' ] = ( ( data_1921[ 'gasto_monetario_deflactado' ]/data_1921[ 'n_personas_hogar' ] ) / 12 ) * 100
data_1921[ 'gasto_bruto_deflactado_pc_m' ]     = ( ( data_1921[ 'gasto_bruto_deflactado' ]/data_1921[ 'n_personas_hogar' ] ) / 12 ) * 100
data_1921[ 'gashog1d_pc_m' ]                   = ( data_1921[ 'gashog1d' ]/data_1921[ 'n_personas_hogar' ] ) / 12
data_1921[ 'gashog2d_pc_m' ]                   = ( data_1921[ 'gashog2d' ]/data_1921[ 'n_personas_hogar' ] ) / 12

## 1.7. Exportar datos

In [25]:
data_1921.to_csv( r'..\..\input\vl\preprocessed_data\base19_21_anualizada_hogares.csv', 
                  index = False )